## 3.7　合并数据集：Concat与Append操作

将不同的数据源进行合并是数据科学中最有趣的事情之一，
* 这既包括将两个不同的数据集非常简单地拼接在一起，
* 也包括用数据库那样的连接（join）与合并（merge）操作处理有重叠字段的数据集。

Series 与DataFrame 都具备这类操作，Pandas 的函数与方法让数据合并变得快速简单。

In [1]:
import pandas as pd
import numpy as np

简单起见，定义一个能够创建DataFrame 某种形式的函数，后面将会用到：

In [3]:
def make_df(cols, ind):
    """一个简单的DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# DataFrame示例
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


### 3.7.1　知识回顾：NumPy数组的合并

合并Series 与DataFrame 与合并NumPy 数组基本相同， 后者通过2.2 节中介绍的`np.concatenate` 函数即可完成。你可以用这个函数将两个或两个以上的数组合并成一个数组。

In [4]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

第一个参数是需要合并的数组列表或元组。还有一个axis 参数可以设置合并的坐标轴方向：

In [5]:
x = [[1, 2],
[3, 4]]
np.concatenate([x, x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

### 3.7.2　通过`pd.concat`实现简易合并

Pandas 有一个`pd.concat()` 函数与`np.concatenate` 语法类似，但是配置参数更多，功能也更强大：
```python
# Pandas 0.18版中的函数签名
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
```
`pd.concat()` 可以简单地合并一维的Series 或DataFrame 对象，与`np.concatenate()` 合并数组一样：

In [6]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

它也可以用来合并高维数据，例如下面的DataFrame：

In [9]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
print("df1: \n", df1, "\n"); print("df2: \n", df2, "\n"); print("pd.concat([df1, df2]): \n", pd.concat([df1, df2]), "\n")

df1: 
     A   B
1  A1  B1
2  A2  B2 

df2: 
     A   B
3  A3  B3
4  A4  B4 

pd.concat([df1, df2]): 
     A   B
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4 



默认情况下，DataFrame 的合并都是**逐行**进行的（默认设置是**axis=0**）。
<br>与`np.concatenate()`一样，`pd.concat` 也可以设置合并坐标轴，例如下面的示例：

In [11]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
print(df3); print(df4); print(pd.concat([df3, df4], axis=1))
# axis='col'会出错

    A   B
0  A0  B0
1  A1  B1
    C   D
0  C0  D0
1  C1  D1
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1


#### 3.7.2.1. 索引重复

`np.concatenate` 与`pd.concat` 最主要的差异之一就是**Pandas在合并时会保留索引**，即使索引是重复的！

In [12]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index # 复制索引
print(x); print(y); print(pd.concat([x, y]))

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
    A   B
0  A0  B0
1  A1  B1
0  A2  B2
1  A3  B3


虽然DataFrame允许这么做，但结果并不是我们想要的。
<br>`pd.concat()` 提供了一些解决这个问题的方法。

**(1) 捕捉索引重复的错误。**

如果你想要检测pd.concat() 合并的结果中是否出现了重复的索引，可以设置`verify_integrity` 参数。将参数设置为True，合并时若有索引重复就会触发异常。
<br>下面的示例可以让我们清晰地捕捉并打印错误信息：

In [13]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


**(2) 忽略索引。**

有时索引无关紧要，那么合并时就可以忽略它们，可以通过设置`ignore_index` 参数来实现。如果将参数设置为True，那么合并时将会创建一个新的整数索引。

In [14]:
print(x); print(y); print(pd.concat([x, y], ignore_index=True))

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
    A   B
0  A0  B0
1  A1  B1
2  A2  B2
3  A3  B3


**(3) 增加多级索引。**

另一种处理索引重复的方法是通过`keys` 参数为数据源设置多级索引标签，这样结果数据就会带上多级索引：

In [15]:
print(x); print(y); print(pd.concat([x, y], keys=['x', 'y']))

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3


示例合并后的结果是多级索引的DataFrame，可以用3.6 节介绍的方法将它转换成我们需要的形式。

#### 3.7.2.2 类似join的合并

前面介绍的简单示例都有一个共同特点，那就是合并的DataFrame 都是同样的列名。而在实际工作中，需要合并的数据往往带有不同的列名，而`pd.concat` 提供了一些选项来解决这类合并问题。
<br>看下面两个DataFrame，它们的列名部分相同，却又不完全相同：

In [18]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
print(df5); print(df6); print(pd.concat([df5, df6]))

    A   B   C
1  A1  B1  C1
2  A2  B2  C2
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4


D:\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  This is separate from the ipykernel package so we can avoid doing imports until


默认情况下，某个位置上缺失的数据会用NaN 表示。
<br>如果不想这样，可以**用`join` 和`join_axes` 参数设置合并方式**。
* 默认的合并方式是对所有输入列进行**并集合并**（`join='outer'`），
* 当然也可以用`join='inner'` 实现对输入列的**交集合并**：

In [19]:
print(df5); print(df6);
print(pd.concat([df5, df6], join='inner'))

    A   B   C
1  A1  B1  C1
2  A2  B2  C2
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4


另一种合并方式是**直接确定结果使用的列名**，设置`join_axes` 参数，里面是索引对象构成的列表（是列表的列表）。如下面示例所示，将结果的列名设置为第一个输入的列名：

In [20]:
print(df5); print(df6);
print(pd.concat([df5, df6], join_axes=[df5.columns]))

    A   B   C
1  A1  B1  C1
2  A2  B2  C2
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
     A   B   C
1   A1  B1  C1
2   A2  B2  C2
3  NaN  B3  C3
4  NaN  B4  C4


pd.concat 的合并功能可以满足你在合并两个数据集时的许多需求，操作时请记住这一点。

#### 3.7.2.3 `append()`方法
因为直接进行数组合并的需求非常普遍，所以Series 和DataFrame 对象都支持append 方法，让你通过最少的代码实现合并功能。
<br>例如，你可以使用`df1.append(df2)`，效果与`pd.concat([df1, df2])` 一样：

In [21]:
print(df1); print(df2); print(df1.append(df2))

    A   B
1  A1  B1
2  A2  B2
    A   B
3  A3  B3
4  A4  B4
    A   B
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4


需要注意的是，与Python列表中的`append()`和`extend()`方法不同，**Pandas的`append()`不直接更新原有对象的值，而是为合并后的数据创建一个新对象**。
<br>因此，它不能被称之为一个非常高效的解决方案，因为每次合并都需要重新创建索引和数据缓存。

总之，如果你需要进行多个`append`操作，还是建议先创建一个DataFrame列表，然后用`concat()`函数一次性解决所有合并任务。